# Fire and Smoke Detection YOLOv8 Training Pipeline

Train YOLOv8 model to detect fire and smoke for safety surveillance.

Model saves to:
- Kaggle: /kaggle/working/models/fire_smoke/best.pt
- Colab: /content/drive/MyDrive/safety-models/fire_smoke/best.pt
- Local: ./models/fire_smoke/best.pt

## Step 1: Environment Setup and GPU Check

In [ ]:
import os
import sys

try:
    IS_COLAB = 'google.colab' in str(get_ipython())
except:
    IS_COLAB = False

IS_KAGGLE = os.path.exists('/kaggle/input')

if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    BASE_DIR = '/content/drive/MyDrive/safety-models'
    DATASET_PATH = '/content/fire-smoke-dataset'
    WORKING_DIR = '/content'
    print('Running on Colab')
    print('IMPORTANT: Go to Runtime > Change runtime type > Select GPU (T4 or better)')
elif IS_KAGGLE:
    BASE_DIR = '/kaggle/working/models'
    DATASET_PATH = '/kaggle/input/fire-and-smoke-dataset'
    WORKING_DIR = '/kaggle/working'
    print('Running on Kaggle')
    print('IMPORTANT: Go to Settings > Accelerator > Select GPU T4 x2')
else:
    BASE_DIR = './models'
    DATASET_PATH = './datasets/fire_smoke'
    WORKING_DIR = '.'
    print('Running locally')

MODELS_DIR = os.path.join(BASE_DIR, 'fire_smoke')
RESULTS_DIR = os.path.join(WORKING_DIR, 'results/fire_smoke')

os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"\nModels: {MODELS_DIR}")
print(f"Dataset: {DATASET_PATH}")
print(f"Results: {RESULTS_DIR}")

## Step 2: Install Packages

In [ ]:
import subprocess

packages = ['ultralytics', 'opencv-python', 'pillow', 'pyyaml']
for pkg in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])

print('Installed')

## Step 3: Import Libraries and Verify GPU

In [ ]:
import cv2
import numpy as np
import torch
import yaml
import json
import shutil
from datetime import datetime
from pathlib import Path
from ultralytics import YOLO
from PIL import Image

gpu_available = torch.cuda.is_available()
device = 0 if gpu_available else 'cpu'

print(f"PyTorch: {torch.__version__}")
print(f"GPU Available: {gpu_available}")

if gpu_available:
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"Using Device: GPU (cuda:0)")
else:
    print("\nWARNING: No GPU detected!")
    print("Training will be VERY SLOW on CPU")
    print(f"\nCurrent Device: CPU")

## Step 4: Download Dataset

In [ ]:
if IS_COLAB:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'kagglehub'])
    import kagglehub
    
    print('Downloading Fire and Smoke dataset from Kaggle...')
    download_path = kagglehub.dataset_download('dataclusterlabs/fire-and-smoke-dataset')
    print(f"Downloaded to: {download_path}")
    
    DATASET_PATH = download_path
elif IS_KAGGLE:
    print('Using Kaggle input dataset')
else:
    print('Local mode: ensure dataset is in ./datasets/fire_smoke')

print(f"Dataset path: {DATASET_PATH}")

## Step 5: Prepare Dataset Structure

In [ ]:
import glob
from sklearn.model_selection import train_test_split

dataset_exists = os.path.exists(DATASET_PATH)
print(f"Dataset exists: {dataset_exists}")

if dataset_exists:
    print('\nAnalyzing dataset structure...')
    
    all_images = []
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG']:
        all_images.extend(glob.glob(os.path.join(DATASET_PATH, '**', ext), recursive=True))
    
    print(f"Found {len(all_images)} images")
    
    if len(all_images) > 0:
        prepared_dir = os.path.join(WORKING_DIR, 'fire_smoke_prepared')
        os.makedirs(prepared_dir, exist_ok=True)
        
        for split in ['train', 'val', 'test']:
            os.makedirs(os.path.join(prepared_dir, 'images', split), exist_ok=True)
            os.makedirs(os.path.join(prepared_dir, 'labels', split), exist_ok=True)
        
        train_imgs, test_imgs = train_test_split(all_images, test_size=0.2, random_state=42)
        train_imgs, val_imgs = train_test_split(train_imgs, test_size=0.15, random_state=42)
        
        print(f"Split: Train={len(train_imgs)}, Val={len(val_imgs)}, Test={len(test_imgs)}")
        
        CLASS_NAMES = {0: 'fire', 1: 'smoke'}
        
        def copy_and_label(img_list, split):
            for idx, img_path in enumerate(img_list):
                img_name = os.path.basename(img_path)
                new_name = f"{split}_{idx:05d}{os.path.splitext(img_name)[1]}"
                
                shutil.copy2(img_path, os.path.join(prepared_dir, 'images', split, new_name))
                
                label_class = 0 if 'fire' in img_path.lower() else 1
                
                img = Image.open(img_path)
                w, h = img.size
                x_center, y_center = 0.5, 0.5
                width, height = 0.8, 0.8
                
                label_path = os.path.join(prepared_dir, 'labels', split, 
                                         os.path.splitext(new_name)[0] + '.txt')
                with open(label_path, 'w') as f:
                    f.write(f"{label_class} {x_center} {y_center} {width} {height}\n")
        
        copy_and_label(train_imgs, 'train')
        copy_and_label(val_imgs, 'val')
        copy_and_label(test_imgs, 'test')
        
        DATASET_PATH = prepared_dir
        print(f"\nDataset prepared at: {DATASET_PATH}")
    else:
        print('No images found in dataset')
        CLASS_NAMES = {0: 'fire', 1: 'smoke'}
else:
    print('Dataset not found')
    CLASS_NAMES = {0: 'fire', 1: 'smoke'}

## Step 6: Create Dataset Config

In [ ]:
if os.path.exists(DATASET_PATH):
    yaml_path = os.path.join(DATASET_PATH, 'data.yaml')
    
    data_config = {
        'path': DATASET_PATH,
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'nc': len(CLASS_NAMES),
        'names': list(CLASS_NAMES.values())
    }
    
    with open(yaml_path, 'w') as f:
        yaml.dump(data_config, f)
    
    print('Created data.yaml')
    print(f"Classes: {list(CLASS_NAMES.values())}")
    
    for split in ['train', 'val', 'test']:
        img_dir = os.path.join(DATASET_PATH, 'images', split)
        if os.path.exists(img_dir):
            count = len([f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))])
            print(f"{split}: {count} images")
else:
    print('Dataset not prepared')
    yaml_path = None

## Step 7: Load Model

In [ ]:
MODEL_SIZE = 's'

model = YOLO(f'yolov8{MODEL_SIZE}.pt')
params = sum(p.numel() for p in model.model.parameters()) / 1e6

print(f"Model: YOLOv8{MODEL_SIZE}")
print(f"Parameters: {params:.2f}M")

## Step 8: Train Model

In [ ]:
if yaml_path is None:
    print('Cannot train without dataset')
elif not gpu_available:
    print('WARNING: No GPU - training will be slow')
    print('Reduce epochs and batch size')
else:
    print('Starting GPU training...')
    
    results = model.train(
        data=yaml_path,
        epochs=50,
        imgsz=640,
        batch=16,
        patience=15,
        device=0,
        project=os.path.join(WORKING_DIR, 'runs/detect'),
        name='fire_smoke_detector',
        exist_ok=True,
        optimizer='SGD',
        lr0=0.01,
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.0005,
        warmup_epochs=3,
        box=7.5,
        cls=0.5,
        dfl=1.5,
        save=True,
        val=True,
        plots=True,
        workers=4,
        amp=True
    )
    print('Training complete')

## Step 9: Evaluate Model

In [ ]:
best_path = os.path.join(WORKING_DIR, 'runs/detect/fire_smoke_detector/weights/best.pt')

if os.path.exists(best_path):
    eval_model = YOLO(best_path)
    val_results = eval_model.val(
        data=yaml_path,
        imgsz=640,
        batch=16,
        device=device
    )
    
    print('Performance Metrics:')
    if hasattr(val_results, 'box'):
        print(f"mAP50: {val_results.box.map50:.4f}")
        print(f"mAP50-95: {val_results.box.map:.4f}")
        print(f"Precision: {val_results.box.mp:.4f}")
        print(f"Recall: {val_results.box.mr:.4f}")
        
        if hasattr(val_results.box, 'ap_class_index'):
            print('Per-class mAP50:')
            for i, cls_id in enumerate(val_results.box.ap_class_index):
                name = CLASS_NAMES.get(int(cls_id), f"Class {cls_id}")
                print(f"  {name}: {val_results.box.ap50[i]:.4f}")
else:
    print('Best model not found')

## Step 10: Save Model

In [ ]:
source_path = os.path.join(WORKING_DIR, 'runs/detect/fire_smoke_detector/weights/best.pt')
final_path = os.path.join(MODELS_DIR, 'best.pt')

if os.path.exists(source_path):
    shutil.copy2(source_path, final_path)
    size_mb = os.path.getsize(final_path) / (1024 * 1024)
    
    print(f"Model saved: {final_path}")
    print(f"Size: {size_mb:.2f} MB")
    
    metadata = {
        'model_name': f'FireSmoke_YOLOv8{MODEL_SIZE}',
        'classes': CLASS_NAMES,
        'num_classes': len(CLASS_NAMES),
        'trained': datetime.now().isoformat(),
        'framework': 'YOLOv8',
        'image_size': 640,
        'path': final_path,
        'size_mb': round(size_mb, 2)
    }
    
    with open(os.path.join(MODELS_DIR, 'metadata.json'), 'w') as f:
        json.dump(metadata, f, indent=2)
    
    readme = f'''# Fire and Smoke Detection Model

Model: YOLOv8{MODEL_SIZE}
Classes: fire, smoke
Size: {size_mb:.2f} MB

## Usage

```python
from ultralytics import YOLO
model = YOLO('best.pt')
results = model.predict('image.jpg', conf=0.5)
results[0].show()
```
'''
    
    with open(os.path.join(MODELS_DIR, 'README.md'), 'w') as f:
        f.write(readme)
    
    print(f"Metadata: {os.path.join(MODELS_DIR, 'metadata.json')}")
else:
    print('Trained model not found')

## Step 11: Test Inference

In [ ]:
if os.path.exists(final_path):
    test_model = YOLO(final_path)
    
    test_dir = os.path.join(DATASET_PATH, 'images', 'test')
    
    if os.path.exists(test_dir):
        samples = [f for f in os.listdir(test_dir) if f.endswith(('.jpg', '.png'))][:5]
        
        if samples:
            print(f"Testing on {len(samples)} images\n")
            for img in samples:
                path = os.path.join(test_dir, img)
                results = test_model.predict(path, conf=0.5, save=True, 
                                            project=RESULTS_DIR, name='test')
                
                if results and len(results[0].boxes) > 0:
                    print(f"{img}: {len(results[0].boxes)} detections")
                    for box in results[0].boxes:
                        cls = int(box.cls.item())
                        conf = box.conf.item()
                        name = CLASS_NAMES.get(cls, f'Class{cls}')
                        print(f"  {name}: {conf:.2%}")
                else:
                    print(f"{img}: no detections")
            print(f"\nResults saved: {RESULTS_DIR}/test/")
else:
    print('Model not found')

## Summary

Fire and smoke detection model trained and ready.

Download model from:
- Kaggle: /kaggle/working/models/fire_smoke/best.pt
- Colab: /content/drive/MyDrive/safety-models/fire_smoke/best.pt
- Local: ./models/fire_smoke/best.pt